A script was created in order to simply acquire all of our data. Each different dataset was uploaded to Google Drive. By modifying the link route below we could access different data versions.

In [8]:
import zipfile
#vanilla training data -- without augmentation
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1AyR_WvrEB5uCUiIfvmwoekVwVCUKxWW7' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1AyR_WvrEB5uCUiIfvmwoekVwVCUKxWW7" -O training_data.zip && rm -rf /tmp/cookies.txt

with zipfile.ZipFile('./training_data.zip', 'r') as zip_ref:
        print('The extraction of the images can take some time...')
        zip_ref.extractall('./')
        print('Extraction done')

--2019-11-27 10:28:40--  https://docs.google.com/uc?export=download&confirm=7_zM&id=1AyR_WvrEB5uCUiIfvmwoekVwVCUKxWW7
Resolving docs.google.com (docs.google.com)... 74.125.142.102, 74.125.142.101, 74.125.142.113, ...
Connecting to docs.google.com (docs.google.com)|74.125.142.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0o-24-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ke2cuoq7stgnsjfpp0fhgm7pn2gtvtge/1574848800000/17506844314399870089/*/1AyR_WvrEB5uCUiIfvmwoekVwVCUKxWW7?e=download [following]
--2019-11-27 10:28:40--  https://doc-0o-24-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ke2cuoq7stgnsjfpp0fhgm7pn2gtvtge/1574848800000/17506844314399870089/*/1AyR_WvrEB5uCUiIfvmwoekVwVCUKxWW7?e=download
Resolving doc-0o-24-docs.googleusercontent.com (doc-0o-24-docs.googleusercontent.com)... 108.177.98.132, 2607:f8b0:400e:c06::84
Connecting to doc-0o-24-docs.googleusercontent.com

ResNet-50 is a convolutional neural network that is trained on more than a million images from the ImageNet database. The network is 50 layers deep and can classify images into 1000 object categories, such as keyboard, mouse, pencil, and many animals. As a result, the network has learned rich feature representations for a wide range of images.

In [5]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import os
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.layers import Dense, Conv2D, Flatten, Dropout, Reshape
from tensorflow.python.keras.layers import Dense
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint 
from sklearn.utils import class_weight

model = Sequential()

model.add(ResNet50(include_top = False, pooling = 'avg', weights = 'imagenet'))
model.add(Dense(256, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(1, activation = 'sigmoid'))
model.layers[0].trainable = False
model.summary()

model.compile(optimizer='adam', metrics=['accuracy'],loss='binary_crossentropy')
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=100)
mc = ModelCheckpoint('model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

base_dir = 'binary_data'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')


image_size = (224,224)
batch_size = 10
seed = 1

train_datagen = ImageDataGenerator(rescale=1./255,
                                  horizontal_flip=True,
                                  rotation_range=75)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=image_size, batch_size=batch_size, class_mode='binary', seed=seed, color_mode="rgb")
validation_generator = test_datagen.flow_from_directory(validation_dir, target_size=image_size, batch_size=batch_size, class_mode='binary', seed=seed, color_mode="rgb")

history = model.fit_generator(train_generator,
                              validation_data=validation_generator,
                              epochs=100, callbacks=[es, mc])
                    
model.save('karibi.h5')

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_9 (Dense)              (None, 256)               524544    
_________________________________________________________________
dense_10 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 129       
Total params: 24,145,281
Trainable params: 557,569
Non-trainable params: 23,587,712
_________________________________________________________________
Found 10097 images belonging to 2 classes.
Found 2526 images belonging to 2 classes.
Epoch 1/100
1009/1010 [============================>.] - ETA: 0s - loss: 0.4634 - accuracy: 0.7819
Epoch 00001: val_accuracy improved from -inf to 0.

KeyboardInterrupt: 

We hoped that the network will provide an efficient base model which we can be developed further by hyperparameter optimization. Sadly its validation accuracy did not improved at all. After five epoch we concluded, that the network is too advanced for our problem.